In [1]:
import pandas as pd
import pyreadr
import numpy as np
import torch
import pickle

In [2]:
patient = pyreadr.read_r('../ServerData_13Oct2020/d_patient_overall.Rdata')
practice = pyreadr.read_r('../ServerData_13Oct2020/d_practice.Rdata')

/opt/conda/envs/rapids/lib/python3.10/site-packages/pyreadr/_pyreadr_parser.py:233: RuntimeWarning: invalid value encountered in cast
  df[colname] = df[colname].values.astype("datetime64[D]").astype(datetime)
/opt/conda/envs/rapids/lib/python3.10/site-packages/pandas/core/tools/datetimes.py:557: RuntimeWarning: invalid value encountered in cast
  arr, tz_parsed = tslib.array_with_unit_to_datetime(arg, unit, errors=errors)
/opt/conda/envs/rapids/lib/python3.10/site-packages/pandas/core/tools/datetimes.py:557: RuntimeWarning: invalid value encountered in multiply
  arr, tz_parsed = tslib.array_with_unit_to_datetime(arg, unit, errors=errors)
/opt/conda/envs/rapids/lib/python3.10/site-packages/pyreadr/_pyreadr_parser.py:233: RuntimeWarning: invalid value encountered in cast
  df[colname] = df[colname].values.astype("datetime64[D]").astype(datetime)
/opt/conda/envs/rapids/lib/python3.10/site-packages/pyreadr/_pyreadr_parser.py:233: RuntimeWarning: invalid value encountered in cast
  df[col

In [3]:
patient = patient['d_patient_overall']
practice = practice['d_practice']

#Age in 2016-01-01
patient['age'] = patient.year_of_birth.apply(lambda x: 2016-x)

In [4]:
patient = patient[['patid', 'practice_id', 'age']].merge(practice[['practice_id', 'Country']], how='left', on='practice_id')

# Clinical

In [5]:
#add paractice information
clinical = pyreadr.read_r('../ServerData_13Oct2020/f_clinical_part1.Rdata')
clinical = clinical['f_clinical_part']

/opt/conda/envs/rapids/lib/python3.10/site-packages/pyreadr/_pyreadr_parser.py:233: RuntimeWarning: invalid value encountered in cast
  df[colname] = df[colname].values.astype("datetime64[D]").astype(datetime)
/opt/conda/envs/rapids/lib/python3.10/site-packages/pandas/core/tools/datetimes.py:557: RuntimeWarning: invalid value encountered in cast
  arr, tz_parsed = tslib.array_with_unit_to_datetime(arg, unit, errors=errors)
/opt/conda/envs/rapids/lib/python3.10/site-packages/pandas/core/tools/datetimes.py:557: RuntimeWarning: invalid value encountered in multiply
  arr, tz_parsed = tslib.array_with_unit_to_datetime(arg, unit, errors=errors)
/opt/conda/envs/rapids/lib/python3.10/site-packages/pandas/core/tools/datetimes.py:557: RuntimeWarning: invalid value encountered in cast
  arr, tz_parsed = tslib.array_with_unit_to_datetime(arg, unit, errors=errors)
/opt/conda/envs/rapids/lib/python3.10/site-packages/pandas/core/tools/datetimes.py:557: RuntimeWarning: invalid value encountered in mu

In [6]:
#data selection 
clinical = clinical.dropna(subset=['code_id'])
clinical['event_date'] = pd.to_datetime(clinical['event_date'])
clinical = clinical.loc[(clinical['event_date'] >= '2016-01-01') & (clinical['event_date'] < '2018-01-01')]

In [7]:
print(clinical.patid.unique().shape)
print(clinical.shape)

(9951,)
(1620518, 11)


In [8]:
#vocab and code2idx generation
vocab = clinical.code_id.unique().tolist()
idx = range(1, len(vocab)+1)

code2idx = dict(zip(vocab, idx))
idx2code = dict(zip(idx, vocab))

code2idx['PAD'] = 0
idx2code[0] = 'PAD'
VOCAB_SIZE = len(code2idx)
print('code2idx Size: {}'.format(len(code2idx)))
print('idx2code Size: {}'.format(len(idx2code)))
# print(code2idx)

code2idx Size: 19860
idx2code Size: 19860


In [9]:
#extract year, month, day from event date
clinical['day'] = clinical.apply(lambda x: str(x['event_date'].day), axis=1)
clinical['month'] = clinical.apply(lambda x: str(x['event_date'].month), axis=1)
clinical['year'] = clinical.apply(lambda x: str(x['event_date'].year), axis=1)

In [10]:
print(clinical.patid.unique().shape)
print(clinical.shape)

(9951,)
(1620518, 14)


In [11]:
# Outcomes data
outcomes = pd.read_csv("../FinalData/cleaned_outcomes_11072023.csv")

In [12]:
outcomes[outcomes.patid.isin(clinical.patid.unique())].outcome_combined_12months.value_counts()

0    6565
1    1505
Name: outcome_combined_12months, dtype: int64

In [13]:
clinical['read_code_seq'] = clinical.sort_values(['event_date'], ascending=True).groupby('patid')['code_id'].transform(lambda x: ', '.join(x))


In [14]:
event_data_seq = clinical.sort_values(['event_date']).groupby('patid').agg({'day': lambda x: x.tolist(),
                                                          'month': lambda x: x.tolist(),
                                                          'year': lambda x: x.tolist()}).reset_index()

In [15]:
clinical = clinical.drop_duplicates(subset=['patid']).reset_index(drop=True)     

In [16]:
print(clinical.patid.unique().shape)
print(clinical.shape)

(9951,)
(9951, 15)


In [17]:
data = clinical[['patid', 'read_code_seq']].merge(event_data_seq, how='left', on='patid')
data = data.merge(outcomes[['patid', 'outcome_combined_12months']], how='inner', on='patid')
data.outcome_combined_12months.value_counts()

0    6565
1    1505
Name: outcome_combined_12months, dtype: int64

In [18]:
print(data.patid.unique().shape)
print(data.shape)

(8070,)
(8070, 6)


In [19]:
data['read_code_seq'] = data['read_code_seq'].apply(lambda x: x.strip('""').split(', '))
data['length_read_code_seq'] = data['read_code_seq'].apply(lambda x: len(x))
data = data[data.length_read_code_seq > 10]

In [20]:
print(data.patid.unique().shape)
print(data.shape)

(7923,)
(7923, 7)


In [21]:
#padding at the beginning of the list
max_seq = 500
def make_uniform_data(x):
    if len(x) < max_seq:
        pads = ['PAD'] * (max_seq - len(x))
        return pads + x
    elif len(x) > max_seq:
        x = x[len(x)-max_seq:]
        return x
    else:
        return x

In [22]:
data['read_code_seq_padded'] = data['read_code_seq'].apply(lambda x: make_uniform_data(x))
data['read_code_seq_padded_idx'] = data['read_code_seq_padded'].apply(lambda x: [code2idx.get(key) for key in x])
data = data.merge(patient[['patid', 'Country', 'age']], how='left', on='patid')

In [23]:
print(data.patid.unique().shape)
print(data.shape)

(7923,)
(7923, 11)


In [24]:
#SAVE all important materials
pickle.dump(code2idx, open('../SeqModel/code2idx.sav', 'wb'))
pickle.dump(idx2code, open('../SeqModel/idx2code.sav', 'wb'))
pickle.dump(data, open('../SeqModel/data.sav', 'wb'))
data.to_csv('../SeqModel/seqDataRaw.csv', index_label=False, index=False)

In [ ]:
data.length_read_code_seq.hist(bins=50)

In [ ]:
data[data.length_read_code_seq > 300].shape

In [ ]:
# a = pickle.load(open('../SeqModel/data.sav', 'rb'))

# Medication

In [ ]:
#add paractice information
therapy = pyreadr.read_r('../ServerData_13Oct2020/f_therapy_part1.Rdata')
therapy = therapy['f_therapy_part']

In [ ]:
#data selection 
therapy = therapy.dropna(subset=['code_id'])
therapy['event_date'] = pd.to_datetime(therapy['event_date'])
therapy = therapy.loc[(therapy['event_date'] >= '2016-01-01') & (therapy['event_date'] < '2018-01-01')]

In [ ]:
therapy.code_id.unique().shape

In [ ]:
therapy.patid.unique().shape

In [ ]:
#vocab and code2idx generation
vocab_therapy = therapy.code_id.unique().tolist()
idx_therapy = range(1, len(vocab_therapy)+1)

code2idx_therapy = dict(zip(vocab_therapy, idx_therapy))
idx2code_therapy = dict(zip(idx_therapy, vocab_therapy))

code2idx_therapy['PAD'] = 0
idx2code_therapy[0] = 'PAD'
VOCAB_SIZE = len(code2idx_therapy)
print('code2idx Size: {}'.format(len(code2idx_therapy)))
print('idx2code Size: {}'.format(len(idx2code_therapy)))
# print(code2idx)

In [ ]:
#extract year, month, day from event date
therapy['day'] = therapy.apply(lambda x: str(x['event_date'].day), axis=1)
therapy['month'] = therapy.apply(lambda x: str(x['event_date'].month), axis=1)
therapy['year'] = therapy.apply(lambda x: str(x['event_date'].year), axis=1)

In [ ]:
therapy['read_code_seq'] = therapy.sort_values(['event_date'], ascending=True).groupby('patid')['code_id'].transform(lambda x: ', '.join(x))
event_data_seq_therapy = therapy.sort_values(['event_date']).groupby('patid').agg({'day': lambda x: x.tolist(),
                                                          'month': lambda x: x.tolist(),
                                                          'year': lambda x: x.tolist()}).reset_index()
therapy = therapy.drop_duplicates(subset=['patid']).reset_index(drop=True)     

In [ ]:
therapy.patid.unique().shape

In [ ]:
therapy.shape

In [ ]:
event_data_seq_therapy.shape

In [ ]:
data_therapy = therapy[['patid', 'read_code_seq']].merge(event_data_seq_therapy, how='left',  on='patid')
data_therapy = data_therapy.merge(outcomes[['patid', 'outcome_combined_12months']], how='inner', on='patid')
data_therapy.outcome_combined_12months.value_counts()

In [ ]:
data_therapy.patid.unique().shape

In [ ]:
data_therapy.shape

In [ ]:
data_therapy['read_code_seq'] = data_therapy['read_code_seq'].apply(lambda x: x.strip('""').split(', '))
data_therapy['length_read_code_seq'] = data_therapy['read_code_seq'].apply(lambda x: len(x))
data_therapy = data_therapy[data_therapy.length_read_code_seq > 10]

In [ ]:
#padding at the beginning of the list
max_seq = 500
def make_uniform_data(x):
    if len(x) < max_seq:
        pads = ['PAD'] * (max_seq - len(x))
        return pads + x
    elif len(x) > max_seq:
        x = x[len(x)-max_seq:]
        return x
    else:
        return x

In [ ]:
data_therapy['read_code_seq_padded'] = data_therapy['read_code_seq'].apply(lambda x: make_uniform_data(x))
data_therapy['read_code_seq_padded_idx'] = data_therapy['read_code_seq_padded'].apply(lambda x: [code2idx_therapy.get(key) for key in x])

In [ ]:
data_therapy = data_therapy.merge(patient[['patid', 'Country', 'age']], how='left', on='patid')

In [ ]:
print(data_therapy.patid.unique().shape)
print(data_therapy.shape)

In [ ]:
#SAVE all important materials
pickle.dump(code2idx_therapy, open('../SeqModel/code2idx_therapy.sav', 'wb'))
pickle.dump(idx2code_therapy, open('../SeqModel/idx2code_therapy.sav', 'wb'))
pickle.dump(data_therapy, open('../SeqModel/data_therapy.sav', 'wb'))
data_therapy.to_csv('../SeqModel/seqData_therapyRaw.csv', index_label=False, index=False)

In [ ]:
data_therapy.length_read_code_seq.hist()